# Projeto 2 RI
<h2>Gonçalo Silva - 103668<br/>
Guilherme Antunes - 103600</h2>

## Install Requirements

In [1]:
!pip install torch transformers ranx scipy==1.8.0 accelerate

## Download and Unzip Files

In [35]:
!wget "https://uapt33090-my.sharepoint.com/:f:/g/personal/guilherme_antunes_ua_pt/EoSzQHtDyzVNrNrcAFEFMZIBbSPWe23aGIbf0c81WLh41w?e=Q4Vvbb&download=1" -O files.zip
!rm -r files
!unzip files
!rm files.zip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Archive:  files.zip
   creating: files/collections/
  inflating: files/collections/pubmed_2022_small.jsonl  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



  inflating: files/collections/pubmed_2022_tiny.jsonl  
   creating: files/dev/
   creating: files/dev/questions_for_train/
  inflating: files/dev/questions_for_train/question_tiny_gs.jsonl  
  inflating: files/dev/questions_for_train/question_small_gs.jsonl  
   creating: files/dev/results/
  inflating: files/dev/results/worst_tiny_results.jsonl  
  inflating: files/dev/results/small_results.jsonl  
  inflating: files/dev/results/worst_small_results.jsonl  
  inflating: files/dev/results/tiny_results.jsonl  
   creating: files/dev/results_filtered/
   creating: files/dev/questions/
  inflating: files/dev/questions/question_E8B2_gs.jsonl  
  inflating: files/dev/questions/question_E8B1_gs.jsonl  
   creating: files/eval/
   creating: files/eval/results/
   creating: files/eval/results/enunciado/
  inflating: files/eval/results/enunciado/BM25_tiny.jsonl  
  inflating: files/eval/results/enunciado/BM25_small.jsonl  
   creating: files/eval/results/b075k12/
  inflating: files/eval/result

## Choose Collection

In [1]:
COLLECTION = "tiny"
# COLLECTION = "small"

## Mine Negative and Positive Examples for Train and Dev set

In [9]:
import json

neg_f = open(f"files/training/worst_BM25_{COLLECTION}.jsonl", "r")
pos_f = open(f"files/training/BM25_{COLLECTION}.jsonl", "r")
f = open("files/training/RI_2023_training_data_wContents.jsonl", "r")

train_set_pos = open(f"files/training/training_set_pos_{COLLECTION}.jsonl", "w")
train_set_neg = open(f"files/training/training_set_neg_{COLLECTION}.jsonl", "w")

for pos, neg, n_pos in zip(f, neg_f, pos_f):
    pos_data = json.loads(pos)
    neg_data = json.loads(neg)
    n_pos_data = json.loads(n_pos)
    query_id = pos_data["id"]
    pos_docs = [d["id"] for d in pos_data["documents"]]
    neg_docs = [d["id"] for d in neg_data["documents"]]
    n_pos_docs = n_pos_data["documents"]
    neg_docs.reverse()

    if len(n_pos_docs) > 0:
        first_doc_score = n_pos_docs[0]["score"]*0.9
        for doc in n_pos_docs:
            if doc["score"] < first_doc_score:
                break
            pos_docs.append(doc["id"])

    if len(pos_docs) == 0 or len(neg_docs) == 0:
        continue
    elif len(pos_docs) < len(neg_docs):
        train_set_pos.write(json.dumps({"query_id": query_id, "pos_docs": pos_docs, "body": pos_data["body"]}) + "\n")
        pos_set = set(pos_docs)
        neg_set = set(neg_docs)
        neg_docs = list(neg_set - pos_set)
        if len(neg_docs) > 0:
            train_set_neg.write(json.dumps({"query_id": query_id, "neg_docs": neg_docs}) + "\n")

pos_f.close()
neg_f.close()
f.close()

neg_f = open(f"files/dev/results/worst_{COLLECTION}_results.jsonl", "r")
pos_f = open(f"files/dev/results/{COLLECTION}_results.jsonl", "r")
f = open(f"files/dev/questions_for_train/question_{COLLECTION}_gs.jsonl", "r")

for pos, neg, n_pos in zip(f, neg_f, pos_f):
    pos_data = json.loads(pos)
    neg_data = json.loads(neg)
    n_pos_data = json.loads(n_pos)
    query_id = pos_data["id"]
    pos_docs = pos_data["documents"]
    neg_docs = [d["id"] for d in neg_data["documents"]]
    n_pos_docs = n_pos_data["documents"]
    neg_docs.reverse()

    if len(n_pos_docs) > 0:
        first_doc_score = n_pos_docs[0]["score"]*0.9
        for doc in n_pos_docs:
            if doc["score"] < first_doc_score:
                break
            pos_docs.append(doc["id"])

    if len(pos_docs) == 0 or len(neg_docs) == 0:
        continue
    elif len(pos_docs) < len(neg_docs):
        train_set_pos.write(json.dumps({"query_id": query_id, "pos_docs": pos_docs, "body": pos_data["query_text"]}) + "\n")
        pos_set = set(pos_docs)
        neg_set = set(neg_docs)
        neg_docs = list(neg_set - pos_set)
        if len(neg_docs) > 0:
            train_set_neg.write(json.dumps({"query_id": query_id, "neg_docs": neg_docs}) + "\n")

pos_f.close()
neg_f.close()
f.close()

train_set_pos.close()
train_set_neg.close()

## Merge Positive, Negative and Dev Docs for Training

In [66]:
import json

training_set_with_pos = open(f"files/training/RI_2023_training_data_wContents.jsonl", "r")
neg_f = open(f"files/training/training_set_neg_{COLLECTION}.jsonl", "r")
pos_f = open(f"files/training/training_set_pos_{COLLECTION}.jsonl", "r")
collection = open(f"files/collections/pubmed_2022_{COLLECTION}.jsonl", "r")
dev_set = open(f"files/dev/questions_for_train/question_{COLLECTION}_gs.jsonl", "r")

with open(f"files/training/training_data_{COLLECTION}.jsonl", "w") as f:
    for line in training_set_with_pos:
        data = json.loads(line)
        documents = data["documents"]
        for doc in documents:
            f.write(json.dumps(doc) + "\n")

    docs = set()
    for line in neg_f:
        data = json.loads(line)
        docs.update(data["neg_docs"])

    for line in pos_f:
        data = json.loads(line)
        docs.update(data["pos_docs"])

    for line in dev_set:
        data = json.loads(line)
        docs.update(data["documents"])
    
    for line in collection:
        data = json.loads(line)
        if data["id"] in docs:
            f.write(line)

## Filter Dev Results

In [67]:
!mkdir files/dev/results_filtered

import json

results = open(f"files/dev/results/{COLLECTION}_results.jsonl", "r")
questions = open(f"files/dev/questions_for_train/question_{COLLECTION}_gs.jsonl", "r")

with open(f"files/dev/results_filtered/{COLLECTION}_results.jsonl", "w") as f:
    with open(f"files/dev/questions_for_train/filtered_question_{COLLECTION}_gs.jsonl", "w") as f2:
        for r, q in zip(results, questions):
            r_data = json.loads(r)
            q_data = json.loads(q)
            r_docs = {doc["id"]: doc["score"] for doc in r_data["documents"]}
            filtered_docs = []
            for doc_id in q_data["documents"]:
                if doc_id in r_docs:
                    filtered_docs.append({"id": doc_id, "score": r_docs[doc_id]})
            if len(filtered_docs) > 0:
                f.write(json.dumps({"id": r_data["id"], "documents": filtered_docs, "question": r_data["question"]}) + "\n")
                f2.write(json.dumps(q_data) + "\n")


mkdir: cannot create directory ‘files/dev/results_filtered’: File exists


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Bert Model

In [68]:
from trainer.data import InferenceRankingIterator,  BioASQPointwiseIterator, InferenceDataset, create_training_dataset
from trainer.sampler import BasicSampler
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.model_max_length = 512

train_ds = create_training_dataset(
    f"files/training/training_set_pos_{COLLECTION}.jsonl",
    f"files/training/training_set_neg_{COLLECTION}.jsonl",
    f"files/training/training_data_{COLLECTION}.jsonl",
    tokenizer=tokenizer,
    iterator_class=BioASQPointwiseIterator[BasicSampler],
)

dev_ds = InferenceDataset(f"files/dev/results_filtered/{COLLECTION}_results.jsonl",
                          train_ds.collection,
                          tokenizer,
                          #max_questions=10, # debug
                          at=100, # max_docs
                          gs_path=f"files/dev/questions_for_train/filtered_question_{COLLECTION}_gs.jsonl",
                          iterator_class=InferenceRankingIterator)


In [69]:
from transformers import AutoModelForSequenceClassification

id2label = {0: "IRRELEVANT", 1: "RELEVANT"}
label2id = {"IRRELEVANT": 0, "RELEVANT": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
).to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
_iter = iter(train_ds)

print(next(_iter))
print(next(_iter))

Token indices sequence length is longer than the specified maximum sequence length for this model (747 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': [101, 2003, 1996, 6121, 3252, 1997, 14255, 2213, 1011, 1015, 2800, 1029, 102, 1037, 12379, 1011, 5250, 8382, 25776, 2817, 1997, 1996, 8031, 1997, 7920, 18595, 2819, 3375, 10099, 2000, 14255, 2213, 2487, 1010, 28177, 2243, 1011, 1017, 29720, 1010, 1998, 3729, 2243, 2475, 1013, 22330, 20464, 2378, 1037, 5250, 21903, 2015, 1012, 2057, 12666, 7241, 25776, 24710, 2000, 2839, 4697, 1996, 10266, 1997, 2048, 4372, 4630, 18994, 22420, 3596, 1997, 1037, 21766, 1011, 3375, 7328, 1006, 1015, 1011, 1054, 1998, 1015, 1011, 1055, 1007, 2007, 2093, 5250, 21903, 2015, 1010, 8419, 14255, 2213, 2487, 1010, 28177, 2243, 1011, 1017, 29720, 1010, 1998, 3729, 2243, 2475, 1013, 22330, 20464, 2378, 1037, 1012, 2057, 2265, 2008, 2256, 7241, 25776, 15078, 8778, 23613, 4275, 1996, 8332, 2838, 1997, 2122, 10266, 1998, 5860, 20026, 28184, 2090, 6637, 23758, 3370, 2389, 12906, 1997, 27854, 1011, 5391, 5250, 5090, 1012, 2478, 1996, 4340, 1060, 1011, 4097, 6121, 3252, 1997, 14255, 2213, 2487, 3375, 2098,

In [72]:
from transformers import TrainingArguments

training_args = TrainingArguments(num_train_epochs=3,
                                  learning_rate=2e-5, # AdamW
                                  weight_decay=0.01,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=32,
                                  evaluation_strategy= "epoch",
                                  dataloader_pin_memory=True,
                                  output_dir="trained_model",
                                  logging_strategy="steps",
                                  logging_first_step=True,
                                  logging_steps=100,
                                  save_strategy="epoch",
                                  save_total_limit=3,
                                  seed=42,
                                  report_to="none")

In [73]:
from trainer.ranker_trainer import RankerTrainer
from transformers import DataCollatorWithPadding
from trainer.collator import RankingCollator
from trainer.metrics import RanxMetrics

import torch

trainer = RankerTrainer(
      model=model,
      args=training_args,
      train_dataset=train_ds,
      eval_dataset=dev_ds, # validation set
      tokenizer=tokenizer,
      data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # apply to train
      eval_data_collator=RankingCollator(tokenizer=tokenizer), # apply to val
      preprocess_logits_for_metrics=lambda logits, labels: torch.nn.functional.softmax(logits, dim=-1)[:,1], # prob for class 1 (relevant class)
      compute_metrics=RanxMetrics(dev_ds.get_qrels()),
  )

In [74]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
!wget "https://uapt33090-my.sharepoint.com/:f:/g/personal/guilherme_antunes_ua_pt/EvrXz1YADDpJqUjgzy0MCr0B-s3lQIvgg6T6CQdW36P1fg?e=CyjlRx&download=1" -O files.zip
!rm -r trained_model
!unzip trained_model
!rm trained_model.zip


## Test Bert Model

In [77]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = "trained_model/checkpoint-2433"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.model_max_length = 512

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).to("cuda")

In [78]:
from trainer.utils import load_collection_lookup
from trainer.data import InferenceRankingIterator, InferenceDataset
from trainer.collator import RankingCollator
import torch
from tqdm import tqdm

# load the collection lookup
collection = load_collection_lookup(f"files/collections/pubmed_2022_{COLLECTION}.jsonl")

eval_ds = InferenceDataset(f"files/eval/results/b075k12/BM25_{COLLECTION}.jsonl",
                            collection,
                            tokenizer,
                            at=100, #max docs pq q
                            iterator_class=InferenceRankingIterator)

dataloader = torch.utils.data.DataLoader(eval_ds,
                                         batch_size=32,
                                         pin_memory=True,
                                         collate_fn=RankingCollator(tokenizer))

In [79]:
_sample = next(iter(dataloader))

_sample["inputs"].input_ids.shape

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([32, 512])

In [80]:
from collections import defaultdict

run_dict = defaultdict(list) # q_id:[{"doc_id": doc_id, "score": prob}]

for sample in tqdm(dataloader):
  _inputs = sample["inputs"].to("cuda") # send inputs to gpu

  # run inference
  with torch.no_grad():
    logits = model(**_inputs).logits # 0->value, 1->value

    # based on the logits, we normalize to have a probabilistic distribution
    # and we extract the probability of that query-doc pair being relevant!
    prob_relevant_pair = torch.nn.functional.softmax(logits, dim=-1)[:,1] # [0-1]
    # 32,

  for i, q_id in enumerate(sample["id"]):
    run_dict[q_id].append({"doc_id":sample["doc_id"][i],
                           "score":prob_relevant_pair[i].item()})


  0%|          | 0/310 [00:00<?, ?it/s]

  0%|          | 1/310 [02:34<13:17:27, 154.85s/it]


KeyboardInterrupt: 

In [8]:
# lest sort by the neural IR model prob and we have our reranked order!
for q_id in run_dict:
  run_dict[q_id].sort(key=lambda x:-x["score"]) #prob


In [11]:
run_dict

[{'doc_id': '21719095', 'score': 0.0018358600791543722},
 {'doc_id': '12392636', 'score': 0.0016552183078601956},
 {'doc_id': '30569273', 'score': 0.0015651995781809092},
 {'doc_id': '10580413', 'score': 0.001373788807541132},
 {'doc_id': '25941654', 'score': 0.0012274904875084758},
 {'doc_id': '32153201', 'score': 0.0012172283604741096},
 {'doc_id': '30528099', 'score': 0.0011905573774129152},
 {'doc_id': '12951644', 'score': 0.0011223664041608572},
 {'doc_id': '31523950', 'score': 0.0011199505534023046},
 {'doc_id': '15793344', 'score': 0.0010940592037513852},
 {'doc_id': '17908555', 'score': 0.0010787309147417545},
 {'doc_id': '23322988', 'score': 0.0010592802427709103},
 {'doc_id': '15567323', 'score': 0.0010372999822720885},
 {'doc_id': '8518456', 'score': 0.0010154175106436014},
 {'doc_id': '20811157', 'score': 0.0010129488073289394},
 {'doc_id': '22997208', 'score': 0.0009966959478333592},
 {'doc_id': '17209507', 'score': 0.0009411661885678768},
 {'doc_id': '24517093', 'score': 

In [ ]:
import json

q = open(f"files/eval/results/b075k12/BM25_{COLLECTION}.jsonl","r")
new_run_dict = run_dict

with open(f"eval_results_{COLLECTION}.jsonl", "w") as f:
    for k, v in new_run_dict.items():
        f.write(json.dumps({"id": k, "documents": [{"id": d["doc_id"], "score": d["score"]} for d in v], "question": json.loads(q.readline())["question"]}) + "\n")

In [8]:
from ranx import evaluate
import json

qrels = {}
with open(f"files/eval/results/b075k12/BM25_{COLLECTION}.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        qrels[data["id"]] = {doc["id"]:1 for doc in data["documents"]}

n_run_dict = {}
with open(f"eval_results_{COLLECTION}.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        n_run_dict[data["id"]] = {doc["id"]:doc["score"] for doc in data["documents"]}

results = evaluate(qrels, run_dict, ["map@10", "recall@10","ndcg@100","mrr@10"])
print(results)

with open(f"eval_{COLLECTION}_metrics.json", "w") as f:
    json.dump(results, f, indent=2)

{'map@10': 0.12894736842105264, 'recall@10': 0.12894736842105264, 'ndcg@100': 1.0, 'mrr@10': 1.0}
